# Introduction

This notebook is summary of Gemini course held by Kaggle.  

First go to [AI Studio](https://aistudio.google.com/app/apikey) and save API KEY locally in a file (`.env`).  

In [ ]:
API_KEY=your_actual_api_key_here

Then you install SDK

In [ ]:
%pip install -U -q "google-generativeai>=0.8.3"

After installing SDK you are ready to load:
- necessary packages 
- load API_KEY

In [1]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API_KEY environment variable
api_key = os.environ.get('API_KEY')
genai.configure(api_key=api_key)

flash = genai.GenerativeModel(model_name='gemini-1.5-flash')

Now try your first question from Gemini!

In [ ]:
flash = genai.GenerativeModel('gemini-1.5-flash')
response = flash.generate_content("Explain AI to me like I'm a kid.")
print(response.text)

You can start a chat by 

In [ ]:
chat = flash.start_chat(history=[])
response = chat.send_message('Hello! My name is Zlork.')
print(response.text)

Check out list of models available in Gemini ([Read more here](https://ai.google.dev/gemini-api/docs/models/gemini))


In [ ]:
for model in genai.list_models():
  print(model.name)

In [ ]:
[model overview page](https://ai.google.dev/gemini-api/docs/models/gemini).